In [4]:
#Object Tracking
import cv2
import numpy as np

# Initalize camera
cap = cv2.VideoCapture(0)

# define range of green color in HSV
lower_green = np.array([45,50,50])
upper_green = np.array([75,200,200])

# Create empty points list
points = []

# Get default camera window size
ret, frame = cap.read()

frame_count = 0 #used to delete trail if object is missing

frame_count1 = 0 #used to delete the trail 

while True:

    # Capture webcame frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv_img, lower_green, upper_green)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        
        # Get the largest contour and its center 
        c = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(c)

        # Allow only countors that have a larger  than pixel 9 radius
        if radius > 9:
            
            # Draw cirlce and leave the last center creating a trail
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 2)
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
            
        # Log center points 
        points.append((int(x), int(y)))
    
        # loop over the set of tracked points
        if radius > 9:
             for i in range(1, len(points)):
                try:
                    cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
                except:
                    pass
        frame_count1+=1
        if frame_count1 >= 30:
            points = []
            frame_count1 = 0
        
    
    else:
        # Count frames 
        frame_count += 1
        
        # If we count 10 frames without object lets delete our trail
        if frame_count >= 10:
             points = []
             # when frame_count reaches 10 let's clear our trail 
             frame_count = 0
    
    # Display our object tracker
    frame = cv2.flip(frame, 1)
    cv2.imshow("Object Tracker", frame)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close any open windows
cap.release()
cv2.destroyAllWindows()